In [1]:
import re
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import requests
import csv
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import googlemaps
import pandas as pd

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}

url = "https://map.kakao.com/"

#현재파일에 있는 크롬 드라이버를 가져와서 열기
options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
# options.add_argument('headless') # 브라우저 안 띄우기
options.add_argument('lang=ko_KR') #  KR 언어
chromedriver_path = r"C:\Users\cxz01\OneDrive\바탕 화면\soongsil university\데이터사이언스\팀프로젝트\DS_Univ_5\web_crawling\chromedriver_win32\chromedriver.exe" # 크롬 드라이버 위치
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

# 1. 카카오 지도로 이동
driver.get(url)

searchloc = '한식'
filename = 'hansic'

#2. 음식점 입력 후 찾기 버튼 클릭 xpath활용
search_area = driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]') # 검색 창
search_area.send_keys(searchloc)  # 검색어 입력
driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색

time.sleep(2)
#3 장소 버튼 클릭
driver.find_element(By.XPATH,'//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER) 

# DataFrame 생성
df = pd.DataFrame(columns=['Addr', 'Score', 'Review'])

def storeNamePrint():
    
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    cafe_lists = soup.select('.placelist > .PlaceItem')
    
    for cafe in cafe_lists:
        
        temp={}
        cafe_name = cafe.select('.head_item > .tit_name > .link_name')[0].text  # cafeName 
        food_score = cafe.select('.rating > .score > .num')[0].text
        review = cafe.select('.rating > .review')[0].text
        link = cafe.select('.contact > .moreview' )[0]['href']
        addr = cafe.select('.addr')[0].text
        
        #"리뷰" 문자열 제거 후 숫자만 반환
        review= review[3:len(review)]
        
        #숫자만 반환된 문자를 쉼표를 
        review = int(re.sub(",","",review))
     
        #temp['Name'] = cafe_name
        temp['Addr'] = addr
        temp['Score'] = food_score
        temp['Review'] = review
        #temp['Link'] = link
        
        df.loc[len(df)] = temp

# 전 음식점 받아오기 위해 
# 페이지 수를 넘기면서 음식점 크롤링

page =1
page2=0

time.sleep(1)

#1페이지부터 34페이지까지 출력
for i in range(0,36):  

    #페이지 넘어가며 출력 
    page2+=1
    print("**",page,"**")

    driver.find_element(By.XPATH,f'//*[@id="info.search.page.no{page2}"]').send_keys(Keys.ENTER)
    storeNamePrint()

    if (page2)%5==0:
        driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
        page2=0

    page+=1
print("**크롤링완료**")

# DataFrame을 csv 파일로 저장
df.to_csv(filename + '.csv', index=False, encoding='utf-8-sig')


C:\Users\cxz01\AppData\Local\Temp\ipykernel_25192\1027172572.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


** 1 **
** 2 **
** 3 **
** 4 **
** 5 **
** 6 **
** 7 **
** 8 **
** 9 **
** 10 **
** 11 **
** 12 **
** 13 **
** 14 **
** 15 **
** 16 **
** 17 **
** 18 **
** 19 **
** 20 **
** 21 **
** 22 **
** 23 **
** 24 **
** 25 **
** 26 **
** 27 **
** 28 **
** 29 **
** 30 **
** 31 **
** 32 **
** 33 **
** 34 **
** 35 **


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x01148893+48451]
	(No symbol) [0x010DB8A1]
	(No symbol) [0x00FE4F20]
	(No symbol) [0x0100CB96]
	(No symbol) [0x0100C401]
	(No symbol) [0x0102A2BC]
	(No symbol) [0x01009586]
	(No symbol) [0x0102A614]
	(No symbol) [0x0103C482]
	(No symbol) [0x0102A0B6]
	(No symbol) [0x01007E08]
	(No symbol) [0x01008F2D]
	GetHandleVerifier [0x013A8E3A+2540266]
	GetHandleVerifier [0x013E8959+2801161]
	GetHandleVerifier [0x013E295C+2776588]
	GetHandleVerifier [0x011D2280+612144]
	(No symbol) [0x010E4F6C]
	(No symbol) [0x010E11D8]
	(No symbol) [0x010E12BB]
	(No symbol) [0x010D4857]
	BaseThreadInitThunk [0x764B7D49+25]
	RtlInitializeExceptionChain [0x7779B74B+107]
	RtlClearBits [0x7779B6CF+191]


In [2]:
# 25개구 for문으로 돌려서 카페 정보 크롤링하기

import os
from time import sleep
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

##########################################################################
##################### variable related selenium ##########################
##########################################################################
# 서울 특별시 구 리스트
gu_list = ['마포구','서대문구','은평구','종로구','중구','용산구','성동구','광진구',
           '동대문구','성북구','강북구','도봉구','노원구','중랑구','강동구','송파구',
           '강남구','서초구','관악구','동작구','영등포구','금천구','구로구','양천구','강서구']


# csv 파일에 헤더 만들어 주기
for index, gu_name in enumerate(gu_list):
    fileName = 'test.csv' # index.__str__() + '_' + gu_name + '.'+'csv'
    file = open(fileName, 'w', encoding='utf-8')
    file.write("카페명" + "|" + "주소" + "|" + "영업시간" + "|" + "전화번호" + "|" + "대표사진주소" + "\n")
    file.close()                                    # 처음에 csv파일에 칼럼명 만들어주기
    
    options = webdriver.ChromeOptions()
    # options.add_argument('headless')
    options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36   ")
    options.add_argument('lang=ko_KR')
    chromedriver_path = r"C:\Users\cxz01\OneDrive\바탕 화면\soongsil university\데이터사이언스\팀프로젝트\DS_Univ_5\web_crawling\chromedriver_win32\chromedriver.exe"
    driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
    driver.get('https://map.kakao.com/')  # 주소 가져오기
    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(gu_name + ' 카페')  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    more_page = driver.find_element_by_id("info.search.place.more")
    # more_page.click()
    more_page.send_keys(Keys.ENTER) # 더보기 누르고
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(1)

    # next 사용 가능?
    next_btn = driver.find_element_by_id("info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while has_next: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
        file = open(fileName, 'a', encoding='utf-8')
        time.sleep(1)
        # place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li:nth-child(1)')
        # 페이지 루프
        #info\.search\.page\.no1 ~ .no5
        page_links = driver.find_elements_by_css_selector("#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        # print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element_by_xpath(xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            sleep(3)
            place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li')
            for p in place_lists: # WebElement
                # print(p.get_attribute('innerHTML'))
                # print("type of p:", type(p))
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                # BS -> 분석
                #
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                # place_address = store_info.select('.info_item > .addr > p')
                # place_hour = store_info.select('.info_item > .openhour > p > a')
                # place_tel = store_info.select('.info_item > .contact > span')
                 # print("length:", len(place_name))
                if len(place_name) == 0:
                    continue # 광고
                place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                place_tel = store_info.select('.info_item > .contact > span')[0].text


                # 사진url 수집
                detail = p.find_element_by_css_selector('div.info_item > div.contact > a.moreview')
                detail.send_keys(Keys.ENTER)

                driver.switch_to.window(driver.window_handles[-1])

                place_photo = ""
                try:
                    photo = driver.find_element_by_css_selector('span.bg_present')
                    photo_url = photo.get_attribute('style')
                    m = re.search('"(.+?)"', photo_url)
                    if m:
                        place_photo = m.group(1)
                    else:
                        place_photo = ""
                except:
                    place_photo = ""
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(place_name, place_photo)

                file.write(place_name + "|" + place_address + "|" + place_hour + "|" + place_tel + "|" + place_photo + "\n")
            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element_by_id("info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            file.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            next_btn.send_keys(Keys.ENTER)
    print("End of Crawl")


C:\Users\cxz01\AppData\Local\Temp\ipykernel_25192\1911719405.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'